In [25]:
import numpy as np
import pandas as pd
import os
import datetime as dt
import talib

In [26]:
# 显示所有列
#pd.set_option('display.max_columns', None)
#pd.set_option('max_colwidth',200)  # 只显示50个
#pd.set_option('display.max_rows', None)  # 行
pd.options.display.max_rows = 20

In [27]:
path_root = os.getcwd()
os.getcwd()
filename = os.listdir(path_root + "/datas/tdx")
maxstocknum = 2  # 股票池最大股票数目
filename = filename[0:maxstocknum]
# 设置测试时间
back_year = 1
end = dt.datetime.now()
sta = dt.datetime.now() - dt.timedelta(days=back_year * 365)
# 创建交易数据集
fname='SH#600519.csv'
df = pd.read_csv(path_root + "/datas/tdx/"+fname,
                     skiprows=[0, 1],  encoding="gbk",skipfooter =1,parse_dates=[0],
                     names=['datetimes', 'open', 'high', 'low', 'close', 'volume', 'amount'],
                     )
df=df.tail(back_year * 365)
df

<ipython-input-27-f11e6532c280>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(path_root + "/datas/tdx/"+fname,


,datetimes,open,high,low,close,volume,amount
1078,2020-02-12,1052.68,1062.47,1049.56,1060.95,2812543,3.072294e+09
1079,2020-02-13,1061.68,1077.57,1051.69,1054.68,3035736,3.335277e+09
1080,2020-02-14,1054.13,1057.19,1046.79,1051.68,2328760,2.534568e+09
1081,2020-02-17,1046.18,1059.87,1046.08,1057.50,2702822,2.946183e+09
1082,2020-02-18,1053.69,1060.56,1046.68,1047.68,2666496,2.897730e+09
...,...,...,...,...,...,...,...
1438,2021-08-02,1664.00,1755.98,1620.72,1755.00,9740065,1.651526e+10
1439,2021-08-03,1730.00,1768.99,1688.80,1749.30,5437475,9.447823e+09
1440,2021-08-04,1734.06,1760.00,1718.00,1724.00,3817549,6.614683e+09
1441,2021-08-05,1692.88,1745.00,1676.36,1699.00,4805969,8.177370e+09


In [28]:
#df['macd'],df['macdsignal'],macdhist=talib.MACDEXT(df['close'],fastperiod=12,slowperiod=26,signalperiod=9)
#df['slowk'],df['slowd']=talib.STOCH(df['high'],df['low'],df['close'],fastk_period=9,slowk_period=3,slowd_period=3)
#df['slowj'] = list(map(lambda x,y: 3*x-2*y, df['slowk'], df['slowd']))


In [29]:
def cross_over(line1,line2):
    up=line1>line2
    down=line1<line2
    up_position=up.shift(1)
    down_position=down.shift(1)
    cross_up=np.where(up & down_position,1,0)
    cross_down=np.where(down & up_position,-1,0)
    cross=cross_up+cross_down
    return cross

#df['macd_cross']=cross_over(df['macd'],df['macdsignal'])


# df1=df.loc[df['macd_cross']!=0][['close','macd','macd_cross']]
# pnl=df1['close']-df1['close'].shift(-1)
# df1['pnl']=pnl

In [30]:
#数据标准版
def stand(factor):
  """自实现标准化
  """
  mean = factor.mean()
  std = factor.std()
  return (factor - mean)/std

# df1['macd_stand']=stand(df1['macd'])
# df1.corr('pearson')
# df1

In [31]:
#macd金叉与收益的散点图
# df1.loc[df1['macd_cross']==1].plot.scatter(x='macd_stand',y='pnl')

In [ ]:
df2=pd.DataFrame()
df2['close']=stand(df['close'])
df2['volume']=stand(df['volume'])
df2['amount']=stand(df['amount'])
df2.corr('pearson')


In [34]:
from sklearn.linear_model import LinearRegression
x=df2['volume']
y=df2['amount']
linreg = LinearRegression()
linreg.fit(x, y)
y_predict = linreg.predict(x)

# 去除线性的关系，留下误差作为该因子的值
df2['pb_ratio'] = y - y_predict
df2

ValueError: Expected 2D array, got 1D array instead:
array=[-0.6008364  -0.45682208 -0.91299528 -0.6716334  -0.69507259 -0.02488754
  0.60430119 -0.54827146  0.07828014  0.06119791  0.39513513 -0.18963037
  0.8071632   0.65981941  0.82675156  1.05535178  1.61242204 -0.43852033
  0.38501251  1.3308678  -0.12192603 -0.05204206  2.48084914  0.89013985
  2.70499523  2.32434932  4.18299171  1.46483758  0.71202366  0.59451539
 -0.02506885 -0.47674727 -0.07588453 -0.43552575  0.68055309 -0.27305676
  0.50125481 -0.63360326  0.39042677 -0.85254738 -0.94336248 -0.62155459
 -1.38442015 -0.04114257 -0.52296428 -1.01364457  0.05216899 -0.85162984
 -0.52993745  0.42570428 -0.68000288 -1.181753   -0.74417901 -0.17907351
 -0.90289977 -0.82438563 -0.103392   -0.61314059 -0.53932836 -0.8882443
 -1.14307633 -0.99515377 -1.21707891 -0.71229291 -0.19841733 -0.81621746
 -1.04876787 -0.92822437 -0.19420065 -0.37915773 -0.96573831 -0.6878168
 -0.70662571 -0.45120909 -0.12630144 -0.74055338 -0.89558913 -1.251